In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


GROUND TRUTH'U 256X256 YAPMAK

In [6]:
import os
import cv2
from google.colab import drive

drive.mount('/content/drive')

input_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth/defect'
output_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect'

os.makedirs(output_dir, exist_ok=True)

target_size = (256, 256)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"⚠️ {filename} yüklenemedi.")
            continue

        resized = cv2.resize(img, target_size, interpolation=cv2.INTER_NEAREST)

        save_path = os.path.join(output_dir, filename)
        cv2.imwrite(save_path, resized)
        print(f"✅ Kaydedildi: {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000000_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000008_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000006_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000007_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100000003_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/101900017_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/101900007_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect/100100002_mask.jpg
✅ Kaydedildi: /content/drive/MyDrive/wood_otsu/

In [25]:
!python /content/drive/MyDrive/EfficientAD/efficientad.py \
  --mvtec_ad_path '/content/drive/MyDrive/wood_otsu' \
  --weights '/content/drive/MyDrive/EfficientAD/models/teacher_small.pth' \
  --output_dir '/content/drive/MyDrive/outputs/efficient' \
  --subdataset wood \
  --model_size small \
  --imagenet_train_path none \
  --train_steps 20000

Computing mean of features: 100% 62/62 [00:00<00:00, 89.53it/s]
Computing std of features: 100% 62/62 [00:00<00:00, 108.98it/s]
Current loss: 4.6216  :  50% 9992/20000 [02:45<02:52, 58.15it/s]
Intermediate map normalization: 100% 7/7 [00:00<00:00, 103.88it/s]

Intermediate inference:   0% 0/141 [00:00<?, ?it/s]
Intermediate inference:  11% 16/141 [00:00<00:00, 158.61it/s]
Intermediate inference:  23% 32/141 [00:00<00:00, 155.73it/s]
Intermediate inference:  35% 49/141 [00:00<00:00, 157.73it/s]
Intermediate inference:  46% 65/141 [00:00<00:00, 150.09it/s]
Intermediate inference:  59% 83/141 [00:00<00:00, 157.30it/s]
Intermediate inference:  70% 99/141 [00:00<00:00, 153.88it/s]
Intermediate inference:  82% 115/141 [00:00<00:00, 153.36it/s]
Intermediate inference: 100% 141/141 [00:00<00:00, 153.92it/s]

✅ Anomaly Detection Sonucları
AUC Score          : 0.9235
F1 Score           : 0.9067
Precision          : 0.8608
Recall             : 0.9577
Optimal Threshold  : 0.2457
Confusion Matrix:


en iyi eşik IoU için

In [26]:
import os
import numpy as np
import cv2
import tifffile as tiff
from glob import glob
from sklearn.metrics import jaccard_score

# Yol tanımları
pred_mask_dir = '/content/drive/MyDrive/outputs/efficient/anomaly_maps/mvtec_ad/wood/test/defect'
gt_mask_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect'

# Dosya yolları
pred_mask_paths = sorted(glob(os.path.join(pred_mask_dir, '*.tiff')))

# Sweep edilecek eşik değerleri
thresholds = np.arange(0.05, 0.95, 0.05)
best_iou = 0
best_threshold = None

# Tüm eşikler için IoU hesapla
for threshold in thresholds:
    ious = []

    for pred_path in pred_mask_paths:
        file_id = os.path.splitext(os.path.basename(pred_path))[0]
        gt_filename = f"{file_id}_mask.jpg"
        gt_path = os.path.join(gt_mask_dir, gt_filename)

        if not os.path.exists(gt_path):
            continue

        # Maskeleri yükle
        pred_mask = tiff.imread(pred_path)
        gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

        # Boyut kontrolü
        if pred_mask.shape != gt_mask.shape:
            gt_mask = cv2.resize(gt_mask, (pred_mask.shape[1], pred_mask.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Binarize et
        pred_bin = (pred_mask > threshold).astype(np.uint8)
        gt_bin = (gt_mask > 0).astype(np.uint8)

        # IoU hesapla
        iou = jaccard_score(gt_bin.flatten(), pred_bin.flatten())
        ious.append(iou)

    # Ortalama IoU hesapla
    if ious:
        mean_iou = np.mean(ious)
        print(f"Eşik: {threshold:.2f} → Ortalama IoU: {mean_iou:.4f}")

        # En iyi sonucu güncelle
        if mean_iou > best_iou:
            best_iou = mean_iou
            best_threshold = threshold

# Sonuç
print(f"\n✅ En iyi eşik: {best_threshold:.2f} ile Ortalama IoU: {best_iou:.4f}")


Eşik: 0.05 → Ortalama IoU: 0.2488
Eşik: 0.10 → Ortalama IoU: 0.3094
Eşik: 0.15 → Ortalama IoU: 0.3210
Eşik: 0.20 → Ortalama IoU: 0.3012
Eşik: 0.25 → Ortalama IoU: 0.2726
Eşik: 0.30 → Ortalama IoU: 0.2428
Eşik: 0.35 → Ortalama IoU: 0.2146
Eşik: 0.40 → Ortalama IoU: 0.1922
Eşik: 0.45 → Ortalama IoU: 0.1716
Eşik: 0.50 → Ortalama IoU: 0.1533
Eşik: 0.55 → Ortalama IoU: 0.1386
Eşik: 0.60 → Ortalama IoU: 0.1268
Eşik: 0.65 → Ortalama IoU: 0.1177
Eşik: 0.70 → Ortalama IoU: 0.1100
Eşik: 0.75 → Ortalama IoU: 0.1025
Eşik: 0.80 → Ortalama IoU: 0.0957
Eşik: 0.85 → Ortalama IoU: 0.0884
Eşik: 0.90 → Ortalama IoU: 0.0812

✅ En iyi eşik: 0.15 ile Ortalama IoU: 0.3210


In [27]:
import os
import numpy as np
import cv2
import tifffile as tiff
from glob import glob
from sklearn.metrics import jaccard_score

# Yol tanımları
pred_mask_dir = '/content/drive/MyDrive/outputs/efficient/anomaly_maps/mvtec_ad/wood/test/defect'
gt_mask_dir = '/content/drive/MyDrive/wood_otsu/wood/ground_truth_resized_256/defect'

# Dosya yollarını al
pred_mask_paths = sorted(glob(os.path.join(pred_mask_dir, '*.tiff')))
gt_mask_paths = sorted(glob(os.path.join(gt_mask_dir, '*_mask.jpg')))

ious = []

# Tahmin maskesi ve ground-truth maske eşlemesi
for pred_path in pred_mask_paths:
    # Dosya adı, örnek: 100000000.tiff -> 100000000_mask.jpg
    file_id = os.path.splitext(os.path.basename(pred_path))[0]
    gt_filename = f"{file_id}_mask.jpg"
    gt_path = os.path.join(gt_mask_dir, gt_filename)

    # Maskeleri yükle
    pred_mask = tiff.imread(pred_path)
    gt_mask = cv2.imread(gt_path, cv2.IMREAD_GRAYSCALE)

    # Maskeleri binary (0 ve 1) hale getir
    pred_bin = (pred_mask > 0.15).astype(np.uint8)
    gt_bin = (gt_mask > 0).astype(np.uint8)

    # Flatten ve IoU hesapla
    iou = jaccard_score(gt_bin.flatten(), pred_bin.flatten())
    ious.append(iou)

    print(f"{file_id} için IoU: {iou:.4f}")

# Ortalama IoU
if ious:
    mean_iou = np.mean(ious)
    print(f"\n🔍 Ortalama IoU: {mean_iou:.4f}")
else:
    print("⚠️ Hiç eşleşen maske bulunamadı.")


100000000 için IoU: 0.5343
100000003 için IoU: 0.2887
100000006 için IoU: 0.2648
100000007 için IoU: 0.6099
100000008 için IoU: 0.2805
100000009 için IoU: 0.0029
100000012 için IoU: 0.3113
100000014 için IoU: 0.6117
100000016 için IoU: 0.2911
100000018 için IoU: 0.3673
100000020 için IoU: 0.0000
100000021 için IoU: 0.4685
100000056 için IoU: 0.4983
100000069 için IoU: 0.1402
100000071 için IoU: 0.0368
100000073 için IoU: 0.4768
100100002 için IoU: 0.6290
100100003 için IoU: 0.5437
100100004 için IoU: 0.4883
100100005 için IoU: 0.3479
100100008 için IoU: 0.3817
100100026 için IoU: 0.5299
100100027 için IoU: 0.4739
100100033 için IoU: 0.6010
100100034 için IoU: 0.3388
100100035 için IoU: 0.4243
100100036 için IoU: 0.4036
100100037 için IoU: 0.4215
100100038 için IoU: 0.2489
100100039 için IoU: 0.4282
100100043 için IoU: 0.4463
100100048 için IoU: 0.3001
100100071 için IoU: 0.0639
100400000 için IoU: 0.3843
100500011 için IoU: 0.1785
100500053 için IoU: 0.0000
100700037 için IoU: 0.2615
1